In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 18 13:04:17 2019

@author: jschmidt
"""
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pandas as pd

import pickle  # pip install dill --user
import matplotlib.pyplot as plt
from keras.applications import VGG16
import tensorflow as tf

import imp
import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)

gpu_fraction = 0.8
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


Using TensorFlow backend.


# Training Phase

In [3]:
COUNTRY = "US"

train_dir = fpr.get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TRAIN")
test_dir = fpr.get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TEST")

# Part II: Pretrained model


conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
        
        
pd_train = pd.read_csv(train_dir + "list.csv")
pd_test = pd.read_csv(test_dir + "list.csv")

#pd_train['orig'] = pd_train['score']
#pd_train['score'] = pd_train['score'] / np.max(pd_train['score'])

#pd_test['orig'] = pd_test['score']
#pd_test['score'] = pd_test['score'] / np.max(pd_test['score'])


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
pd_train.head()

,Unnamed: 0,id,t_hh,id_s,score
0,5,6,23.0,6.png,23.0
1,23,24,30.0,24.png,30.0
2,25,26,60.0,26.png,60.0
3,29,30,30.0,30.png,30.0
4,30,31,24.6,31.png,24.6


In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   horizontal_flip = True,
                                   fill_mode = "nearest", 
                                   zoom_range = 0.2,
                                   width_shift_range = 0.2, 
                                   height_shift_range = 0.2,
                                   rotation_range = 30,
                                   validation_split = 0.25) 

train_generator = train_datagen.flow_from_dataframe(dataframe = pd_train, 
                                                    directory = train_dir, 
                                                    x_col = "id_s", 
                                                    y_col = "score", 
                                                    has_ext = True, 
                                                    class_mode="other", 
                                                    subset="training",
                                                    batch_size=20)

#                                                    target_size=(img_width, img_height), 


val_generator = train_datagen.flow_from_dataframe(dataframe = pd_train, 
                                                    directory = train_dir, 
                                                    x_col = "id_s", 
                                                    y_col = "score", 
                                                    has_ext = True, 
                                                    class_mode="raw", 
                                                    subset="validation",
                                                    batch_size=20)

#                                                    target_size=(img_width, img_height), 

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe = pd_test, 
                                                    directory = test_dir, 
                                                    x_col = "id_s", 
                                                    y_col = "score", 
                                                    has_ext = True, 
                                                    class_mode="raw", 
                                                    batch_size=20)

Found 27389 validated image filenames.
Found 9129 validated image filenames.
Found 9128 validated image filenames.


In [11]:
model.compile(loss='mean_squared_error',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

mcp_save = ModelCheckpoint('models/model-regression-unfreezed-2-layer-2-layer-classification-6497-turbines-{epoch:04d}-{val_loss:.4f}.h5',
                           save_best_only=True, monitor='val_loss', mode='min')




In [13]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=val_generator,
      validation_steps=50,
      callbacks=[mcp_save],
      verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
 - 78s - loss: 488.3208 - acc: 0.0255 - val_loss: 200.7233 - val_acc: 0.0280
Epoch 2/2
 - 75s - loss: 172.3101 - acc: 0.0300 - val_loss: 343.2137 - val_acc: 0.0170


# Validation vs. Training

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()




# Model selection

In [ ]:
min_val_loss = min(val_loss)

i = val_loss.index(min_val_loss) + 1

best_model = "models/model-unfreezed-2-layer-2-layer-classification-6497-turbines-" + "{0:0=4d}".format(i) + "-" + "{0:.4f}".format(min_val_loss) + ".h5"

#best_model = "models/model-unfreezed-2-layer-2-layer-classification-6497-turbines-0005-0.0339.h5"

best_model

# Test

In [ ]:
test_base_dir = fpr.get_param(COUNTRY, "PATH_ML_IMAGES_TURBINES_TEST")+"../"

test_model = models.load_model(best_model)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_base_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary')

test_loss, test_acc = test_model.evaluate_generator(test_generator, steps=32)
print('test acc:', test_acc)
print('test loss:', test_loss)
